In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("de1-check").getOrCreate()
print("Spark:", spark.version)
print("PySpark:", pyspark.__version__)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/15 10:49:47 WARN Utils: Your hostname, OrdideJustine, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/15 10:49:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/15 10:49:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark: 4.0.1
PySpark: 4.0.1


In [6]:
import os, csv, pathlib
pathlib.Path("Data_engineering/data").mkdir(exist_ok=True)
rows = [
    {"user_id":1,"product_id":101,"price":9.9,"ts":"2025-09-01T09:00:00"},
    {"user_id":1,"product_id":102,"price":19.0,"ts":"2025-09-01T09:02:00"},
    {"user_id":2,"product_id":101,"price":9.9,"ts":"2025-09-02T10:00:00"},
    {"user_id":3,"product_id":103,"price":5.5,"ts":"2025-09-03T11:30:00"},
]
with open("Data_engineering/data/sample_sales.csv","w",newline="") as f:
    w = csv.DictWriter(f, fieldnames=["user_id","product_id","price","ts"])
    w.writeheader(); w.writerows(rows)
print("Wrote Data_engineering/data/sample_sales.csv, bytes:", os.path.getsize("Data_engineering/data/sample_sales.csv"))
df = spark.read.option("header","true").option("inferSchema","true").csv("Data_engineering/data/sample_sales.csv")
df.printSchema()
df.show()

Wrote Data_engineering/data/sample_sales.csv, bytes: 154
root
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- ts: timestamp (nullable = true)

+-------+----------+-----+-------------------+
|user_id|product_id|price|                 ts|
+-------+----------+-----+-------------------+
|      1|       101|  9.9|2025-09-01 09:00:00|
|      1|       102| 19.0|2025-09-01 09:02:00|
|      2|       101|  9.9|2025-09-02 10:00:00|
|      3|       103|  5.5|2025-09-03 11:30:00|
+-------+----------+-----+-------------------+



In [10]:
df = spark.read.option("header","true").option("inferSchema","true").csv("Data_engineering/data/sample_sales.csv")
df.printSchema()
df.show()
df.explain("formatted")

root
 |-- user_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- ts: timestamp (nullable = true)

+-------+----------+-----+-------------------+
|user_id|product_id|price|                 ts|
+-------+----------+-----+-------------------+
|      1|       101|  9.9|2025-09-01 09:00:00|
|      1|       102| 19.0|2025-09-01 09:02:00|
|      2|       101|  9.9|2025-09-02 10:00:00|
|      3|       103|  5.5|2025-09-03 11:30:00|
+-------+----------+-----+-------------------+

== Physical Plan ==
Scan csv  (1)


(1) Scan csv 
Output [4]: [user_id#251, product_id#252, price#253, ts#254]
Batched: false
Location: InMemoryFileIndex [file:/mnt/c/Users/Justine/Data_engineering/data/sample_sales.csv]
ReadSchema: struct<user_id:int,product_id:int,price:double,ts:timestamp>




In [14]:
from pyspark.sql import functions as F
agg = df.groupBy("user_id").agg(F.count("*").alias("n"), F.sum("price").alias("total"))
agg.show()
plan = agg._jdf.queryExecution().executedPlan().toString()
print(plan)
print("\n=== explain formatted ===")
agg.explain("formatted")

# Save evidence
import pathlib, datetime
pathlib.Path("Data_engineering/proof").mkdir(exist_ok=True)
with open("Data_engineering/proof/plan_formatted.txt","w") as f:
    f.write(str(datetime.datetime.now()) + "\n\n")
    f.write(str(plan))
print("Saved Data_engineering/proof/plan_formatted.txt")

+-------+---+-----+
|user_id|  n|total|
+-------+---+-----+
|      1|  2| 28.9|
|      3|  1|  5.5|
|      2|  1|  9.9|
+-------+---+-----+

AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[user_id#251], functions=[count(1), sum(price#253)], output=[user_id#251, n#351L, total#352])
   +- Exchange hashpartitioning(user_id#251, 200), ENSURE_REQUIREMENTS, [plan_id=441]
      +- HashAggregate(keys=[user_id#251], functions=[partial_count(1), partial_sum(price#253)], output=[user_id#251, count#364L, sum#365])
         +- FileScan csv [user_id#251,price#253] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/mnt/c/Users/Justine/Data_engineering/data/sample_sales.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:int,price:double>


=== explain formatted ===
== Physical Plan ==
AdaptiveSparkPlan (5)
+- HashAggregate (4)
   +- Exchange (3)
      +- HashAggregate (2)
         +- Scan csv  (1)


(1) Scan csv 
Output [2]: [user

In [15]:
spark.stop()
print("Spark session stopped.")

Spark session stopped.
